# 1. Loading Data

In [1]:
import sys
import gc
import os
import warnings
import pickle
import statsmodels.api as sm
from pylab import rcParams
import time

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, metrics

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

register_matplotlib_converters()
sns.set()

Reduced = False

/usr/local/anaconda3/envs/data_science/lib/python3.8/site-packages/lightgbm/__init__.py:42: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  warnings.warn("Starting from version 2.2.1, the library file in distribution wheels for macOS "


## 1.1 Functions

In [2]:
def print_size_var():
    print("Size of Variables")
    _vars = globals().items()
    for var, obj in _vars:
        print(var, sys.getsizeof(obj))
    
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_data():
    print('Reading files...')
    INPUT_DIR = 'data'
    calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
    sales_train_validation = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
    sell_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
    sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
    
    calendar = reduce_mem_usage(calendar)
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    sell_prices = reduce_mem_usage(sell_prices)
    sample_submission = reduce_mem_usage(sample_submission)

    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    print('Sample_submission has {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return calendar, sell_prices, sales_train_validation, sample_submission

def display_missing(df):    
    for col in df.columns.tolist():  
        if df[col].isnull().sum() != 0:
            print('{} column missing values: {}'.format(col, df[col].isnull().sum()))
    print('\n')
    
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null)+1, index=not_null.index)

    return df    

def melt(sales_train_validation, submission, reduced):
    # melt sales data, get it ready for training
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    # get product table.
    product = sales_train_validation[id_columns]
    
    if reduced == True :
        sales_train_validation = sales_train_validation.drop([f"d_{d}" for d in range(1, 719)], axis = 1).melt( id_vars=id_columns, var_name="d", value_name="demand" )
    else:
        sales_train_validation = sales_train_validation.melt(id_vars=id_columns, var_name="d", value_name="demand")

    sales_train_validation = reduce_mem_usage(sales_train_validation, verbose=False)

    # separate test dataframes.
    vals = submission[submission["id"].str.endswith("validation")]
    evals = submission[submission["id"].str.endswith("evaluation")]

    # change column names.
    vals.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + 28)]
    evals.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + 28)]

    # merge with product table
    evals["id"] = evals["id"].str.replace("_evaluation", "_validation")
    vals = vals.merge(product, how="left", on="id") # Adds info [item_id;dept_id;cat_id;store_id;state_id]
    evals = evals.merge(product, how="left", on="id")
    evals["id"] = evals["id"].str.replace("_validation", "_evaluation")
    
    del product
    gc.collect()
    
    vals = vals.melt(id_vars=id_columns, var_name="d", value_name="demand")
    evals = evals.melt(id_vars=id_columns, var_name="d", value_name="demand")

    sales_train_validation["part"] = "train"
    vals["part"] = "validation"
    evals["part"] = "evaluation"

    data = pd.concat([sales_train_validation, vals, evals], axis=0)
    del sales_train_validation, vals, evals

    # delete evaluation for now.
    data = data[data["part"] != "evaluation"]
    gc.collect()
    return data


def extract_d(df):
    return df["d"].str.extract(r"d_(\d+)").astype(np.int16)


def merge_calendar(data, calendar):
    #calendar = calendar.drop(["weekday", "wday", "month", "year", 'event_name_1', 'event_name_2'], axis=1)
    return data.merge(calendar, how="left", on="d").assign(d=extract_d)


def merge_sell_prices(data, sell_prices):
    return data.merge(sell_prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])

## 1.2 Load csv files to variables

In [3]:
#Load csv files to variables
calendar, sell_prices, sales_train_validation, sample_submission = read_data()

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
Calendar has 1969 rows and 14 columns
Sales train validation has 30490 rows and 1919 columns
Sell prices has 6841121 rows and 4 columns
Sample_submission has 60980 rows and 29 columns


 ## 1.3 Encode categorical variables

In [4]:
calendar=reduce_mem_usage(calendar)

sales_train_validation = encode_categorical(
    sales_train_validation, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]
).pipe(reduce_mem_usage)

product2 = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

Mem. usage decreased to  0.12 Mb (0.0% reduction)
Mem. usage decreased to 94.01 Mb (1.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)


## 1.4 Melt 

In [5]:
data = melt(sales_train_validation, sample_submission ,Reduced)
del sales_train_validation
del sample_submission

data = reduce_mem_usage(data)
data1 = data.loc[:len(data["part"])/2]
data2 = data.loc[len(data["part"])/2:]
del data
gc.collect()

print("Time-related data")
data1 = merge_calendar(data1, calendar)
data2 = merge_calendar(data2, calendar)
del calendar
gc.collect()

frames = [data1, data2]
data = pd.concat(frames)
del data1,data2,frames
gc.collect()

print("Price-related data")
data = merge_sell_prices(data, sell_prices)
del sell_prices
gc.collect()
data = reduce_mem_usage(data)

Mem. usage decreased to 2257.58 Mb (0.0% reduction)
Time-related data
Price-related data
Mem. usage decreased to 5248.87 Mb (0.0% reduction)


## 1.5 Combine SNAP

In [6]:
conditions = [
    (data['state_id'] == 1),
    (data['state_id'] == 2),
    (data['state_id'] == 3)]
choices = [data['snap_CA'], data['snap_TX'], data['snap_WI']]
data['snap'] = np.select(conditions, choices)
data =data.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis=1)
data = reduce_mem_usage(data)
del conditions, choices
gc.collect()


Mem. usage decreased to 5135.99 Mb (0.0% reduction)


20

## 1.6 Remove rows with NA values

In [ ]:
#Remove na values of sale_price
data.dropna(subset=['sell_price'],inplace=True)
data = reduce_mem_usage(data)

## 1.7 Add Calendar Feutures

In [8]:
data['woy']   = data['wm_yr_wk']%100
data['fyear'] = (data['wm_yr_wk']//100)%100
data['week_of_total'] = np.where( data['fyear'] <14, (data['woy'] + (data['fyear']- 11) * 52 ), ( 157+ data['woy']  + ((data['fyear']- 14) * 52)) ) 
data["is_weekend"] = data["wday"].isin([1, 2]).astype(np.int8)
data = reduce_mem_usage(data)

Mem. usage decreased to 4292.15 Mb (2.0% reduction)


## 1.8 Setting data types for optimal memory usage

In [9]:
data['id'] = data['id'].astype('category') 
data['part'] = data['part'].astype('category')
data['date'] = data['date'].astype('datetime64[ns]')
data['weekday'] = data['weekday'].astype('category')
data['event_name_1'] = data['event_name_1'].astype('category')
data['event_type_1'] = data['event_type_1'].astype('category')
data['event_name_2'] = data['event_name_2'].astype('category')
data['event_type_2'] = data['event_type_2'].astype('category')

In [10]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price,snap,woy,fyear,week_of_total,is_weekend
7,HOBBIES_1_008_CA_1_validation,1445,4,2,1,1,1,12,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0.459961,0,1,11,1,1
8,HOBBIES_1_009_CA_1_validation,1446,4,2,1,1,1,2,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,1.559570,0,1,11,1,1
9,HOBBIES_1_010_CA_1_validation,1447,4,2,1,1,1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,3.169922,0,1,11,1,1
11,HOBBIES_1_012_CA_1_validation,1449,4,2,1,1,1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,5.980469,0,1,11,1,1
14,HOBBIES_1_015_CA_1_validation,1452,4,2,1,1,1,4,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0.700195,0,1,11,1,1


In [11]:
#print('Data usage: {} GB'.format(data.memory_usage().sum() / 10**9))
print('Data usage: {}'.format(data.memory_usage(deep=True) / 10**6))


Data usage: Index            375.053424
id                97.688416
item_id           93.763356
dept_id           46.881678
cat_id            46.881678
store_id          46.881678
state_id          46.881678
d                 93.763356
demand            93.763356
part              46.881887
date             375.053424
wm_yr_wk          93.763356
weekday           46.882447
wday              46.881678
month             46.881678
year              93.763356
event_name_1      46.885011
event_type_1      46.882099
event_name_2      46.882111
event_type_2      46.881889
sell_price        93.763356
snap              46.881678
woy               46.881678
fyear             46.881678
week_of_total     93.763356
is_weekend        46.881678
dtype: float64


## 1.9 Saving the data

In [12]:
print("Saving the data...")
data.to_pickle('m5_data.pkl')
# data.to_hdf('m5_data.h5', key='df', mode='w', format='table')

Saving the data...
